In [29]:
#sklearn imports
import sklearn
from sklearn import preprocessing, neighbors
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import scale

import pickle
import pandas as pd
import numpy as np
import random, os
import csv
from typing import Tuple

with open('kmeans_5clusters.pkl', 'rb') as f:
    clustering = pickle.load(f)
    
with open('clusterI_regression_model.pkl', 'rb') as f:
    sgdc0 = pickle.load(f)

In [30]:
def scale_features_single(X: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    applies standard scaler (z-scores) to training data and predicts z-scores for the test set
    """
    scaler = StandardScaler()
    to_scale = [col for col in X.columns.values]
    scaler.fit(X[to_scale])
    X[to_scale] = scaler.transform(X[to_scale])
    
    return X
      
def classify_ext_data(input_csv, subset = "none"):      
    ext_df = pd.read_csv(input_csv)
    ext_df.fillna(-99999)
    receptors = ext_df.Receptor
    hits_actual = ext_df.Hits
    score_types = ext_df['Score Type']
    subsets = ext_df.subset
    match_features = ext_df.match_features
    init_ext_df = ext_df
    
    #check if a 'quality' column exists. one will not exist if classifying data with unknown enrichments.
    if 'quality' not in ext_df:
        ext_df = ext_df[['s_score','Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop']]
        x = ext_df
    else:
        ext_df = ext_df[['s_score','Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop', 'quality']]   
        x = ext_df.drop('quality', axis=1)
        y = ext_df.quality
    
    predictors = list(ext_df.columns)
    predictors = predictors[:-1]
    print('Predictors:', predictors,'\n')
    
    if 'quality' in init_ext_df:
        print("score based Q ph4s:", y.sum(),'\n')
    
    # predict labels on the external set
    ext_labels = clustering.predict(x)
    X_clstrs = x.copy()
    
    X_scaled = scale_features_single(X_clstrs)
    ext_clusters = X_scaled.copy()
    if 'quality' in init_ext_df:
        ext_clusters['y'] = y
    
    #add receptors, hits_actual, score type, and subset columns back prior to 0/1/2/3 split
    ext_clusters['Receptor'] = receptors
    ext_clusters['hits_actual'] = hits_actual
    ext_clusters['Score Type'] = score_types
    ext_clusters['subset'] = subsets
    ext_clusters['match_features'] = match_features
    ext_clusters['clusters'] = ext_labels
    
    uniq_clusters = ext_clusters['clusters'].unique()
    uniqs = uniq_clusters.tolist()
    uniqs.sort()
    print(uniqs)
    
    #write ext_clusters to csv
    #X_train_clstrs.to_csv('X_train_clstrs2.csv')
    
    # locate the "0" cluster
    ext_0 = ext_clusters.loc[ext_clusters.clusters == 0]
    if 'quality' in init_ext_df:
        y_ext_0 = ext_0.y.values
    ext_0_receptors = ext_0.Receptor
    ext_0_hits_actual = ext_0.hits_actual
    ext_0_score_types = ext_0['Score Type']
    ext_0_subsets = ext_0.subset
    ext_0_match_features = ext_0.match_features
    
    # locate the "1" cluster
    ext_1 = ext_clusters.loc[ext_clusters.clusters == 1]
    if 'quality' in init_ext_df:
        y_ext_1 = ext_1.y.values
    ext_1_receptors = ext_1.Receptor
    ext_1_hits_actual = ext_1.hits_actual
    ext_1_score_types = ext_1['Score Type']
    ext_1_subsets = ext_1.subset
    ext_1_match_features = ext_1.match_features
    
    # locate the "2" cluster
    ext_2 = ext_clusters.loc[ext_clusters.clusters == 2]
    if 'quality' in init_ext_df:
        y_ext_2 = ext_2.y.values
    ext_2_receptors = ext_2.Receptor
    ext_2_hits_actual = ext_2.hits_actual
    ext_2_score_types = ext_2['Score Type']
    ext_2_subsets = ext_2.subset
    ext_2_match_features = ext_2.match_features
    
    # locate the "3" cluster
    ext_3 = ext_clusters.loc[ext_clusters.clusters == 3]
    if 'quality' in init_ext_df:
        y_ext_3 = ext_3.y.values
    ext_3_receptors = ext_3.Receptor
    ext_3_hits_actual = ext_3.hits_actual
    ext_3_score_types = ext_3['Score Type']
    ext_3_subsets = ext_3.subset
    ext_3_match_features = ext_3.match_features

    # drop the targets from each external set (if classifying known external data)
    if 'quality' in init_ext_df:
        X_ext_0 = ext_0.drop(columns=['y', 'Receptor', 'hits_actual', 'Score Type', 'subset', 'match_features'])
        X_ext_1 = ext_1.drop(columns=['y', 'Receptor', 'hits_actual', 'Score Type', 'subset', 'match_features'])
        X_ext_2 = ext_2.drop(columns=['y', 'Receptor', 'hits_actual', 'Score Type', 'subset', 'match_features'])
        X_ext_3 = ext_3.drop(columns=['y', 'Receptor', 'hits_actual', 'Score Type', 'subset', 'match_features'])
    else:
        X_ext_0 = ext_0.drop(columns=['Receptor', 'hits_actual', 'Score Type', 'subset', 'match_features'])
        X_ext_1 = ext_1.drop(columns=['Receptor', 'hits_actual', 'Score Type', 'subset', 'match_features'])
        X_ext_2 = ext_2.drop(columns=['Receptor', 'hits_actual', 'Score Type', 'subset', 'match_features'])
        X_ext_3 = ext_3.drop(columns=['Receptor', 'hits_actual', 'Score Type', 'subset', 'match_features'])
    
    #predict based on 0 cluster model
    print('0 cluster model\n')
    print('---------------\n')
    if len(X_ext_0) == 0:
        print('No cluster 0 data.\n')
    else:
        y_pred = (sgdc0.predict(X_ext_0))
        if 'quality' in init_ext_df:
            confmat = confusion_matrix(y_ext_0, y_pred, labels=[0,1])
            confmat = confusion_matrix(y_ext_0, y_pred, labels=[0,1]).ravel()
            FP = (confmat[1])
            TP = (confmat[3])
            PPV = (TP / (TP + FP))
            cm = pd.crosstab(y_ext_0, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=False)
            print(cm,'\n')
            print('PPV:', format(PPV, '.2f'))
            print("Accuracy:",format(metrics.accuracy_score(y_ext_0, y_pred),'.2f'))
            print("Recall:",format(metrics.recall_score(y_ext_0, y_pred), '.2f'))
            print("F1:",format(metrics.f1_score(y_ext_0, y_pred), '.2f'),'\n')
        
        X_ext_0['Receptor'] = ext_0_receptors
        X_ext_0['hits_actual'] = ext_0_hits_actual
        X_ext_0['Score Type'] = ext_0_score_types
        X_ext_0['subset'] = ext_0_subsets
        X_ext_0['match_features'] = ext_0_match_features
        if 'quality' in init_ext_df:
            X_ext_0['quality'] = y_ext_0
        
        X_ext_0['quality_pred'] = y_pred
        if 'quality' in init_ext_df:
            X_ext_0.to_csv('results/'+subset+'/0cluster_results.csv')
        #if classifying unknown external data, print the ph4s classified as quality and write them to .csv
        if 'quality' not in init_ext_df:
            print(X_ext_0.loc[X_ext_0['quality_pred'] == 1], '\n')
            ph4_preds = X_ext_0.loc[X_ext_0['quality_pred'] == 1]
            ph4_preds.to_csv('predictions/0cluster_ph4_preds.csv')

In [31]:
classify_ext_data('data/gpr37_data_binary_5feats.csv')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop'] 

[0, 2]
0 cluster model

---------------

     s_score      Hits  max_feat  avg_feat  max_centr  min_centr  avg_centr  \
12  0.434497 -0.726544  1.320207  0.510117   0.634537   1.346594   2.363602   
16  0.132312 -0.751048  1.558654  1.630742   0.618431   0.625537   1.083783   

    features  all_same  hyd_prop  ...  catdon_prop  hydaro_prop  aniacc_prop  \
12       0.0 -0.229416  0.315058  ...    -0.479616    -0.714545          0.0   
16       0.0 -0.229416  0.315058  ...     0.319744    -0.714545          0.0   

    clusters  Receptor hits_actual  Score Type  subset match_features  \
12         0     GPR37           2   dE(class)  rec_ef              5   
16         0     GPR37           1   dE(class)  rec_gh              5   

    quality_pred  
12             1  
16             1  

[2 rows x 21 columns] 



In [32]:
classify_ext_data('data/gpr37_data_binary_6feats.csv')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop'] 

[0, 2]
0 cluster model

---------------

     s_score      Hits  max_feat  avg_feat  max_centr  min_centr  avg_centr  \
0   1.552769  0.917663  0.192155  0.452455   0.605285   0.435928   0.966185   
2   0.810465  0.630893 -0.010452  0.392294   0.605285  -1.753278  -0.726884   
4   0.339387  0.344124  0.308863  0.897490   0.605285   0.435928   0.980503   
12  0.434497 -0.516185  1.320207  0.510117   0.634537   1.346594   2.363602   
14  1.025880 -0.516185  1.312671  1.360411   0.567956   1.261856   1.468969   
16  0.132312 -0.516185  1.558654  1.630742   0.618431   0.625537   1.083783   
17  0.062475 -0.516185  1.249371  1.116749   0.955636  -0.913274  -0.027683   
18 -0.183173 -0.516185  1.054133  0.767183   0.605285  -1.753278   0.036188   

    features  all_same  hyd_prop  ...  catdon_prop  hydaro_prop  aniacc_

In [33]:
classify_ext_data('data/gpr37_data_binary_6feats_ZINC.csv')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop'] 

[0, 2]
0 cluster model

---------------

     s_score      Hits  max_feat  avg_feat  max_centr  min_centr  avg_centr  \
12  0.434497 -0.404446  1.320207  0.510117   0.634537   1.346594   2.363602   
16  0.132312 -0.404446  1.558654  1.630742   0.618431   0.625537   1.083783   

    features  all_same  hyd_prop  ...  catdon_prop  hydaro_prop  aniacc_prop  \
12       0.0 -0.229416  0.315058  ...    -0.479616    -0.714545          0.0   
16       0.0 -0.229416  0.315058  ...     0.319744    -0.714545          0.0   

    clusters  Receptor hits_actual  Score Type  subset match_features  \
12         0     GPR37           0   dE(class)  rec_ef              6   
16         0     GPR37           0   dE(class)  rec_gh              6   

    quality_pred  
12             1  
16             1  

[2 rows x 21 columns] 

